### This notebook covers data scraping, cleaning, processing and updating of the [COVID-19 Singapore dataset](https://data.world/hxchua/covid-19-singapore/workspace/file?filename=Covid-19+SG.xlsx).

Steps to run:
1. Update `phase` (This phase will be inputted into the Google Sheets.)
2. Restart and run all.

Additional steps if you want to update a Google Sheet:
1. Start a Google Cloud Platform Project. Get/download the service account credentials, copy the downloaded json into the same directory as this file and update the `KEYS_FILE` variable with the json name.
2. Update Config for Google Sheets
3. Set `update_google_sheets` to True

## Import required packages

In [1]:
import sys
from bs4 import BeautifulSoup
import requests
import urllib3
import io
import pandas as pd
from datetime import datetime, timedelta
import re
from google.oauth2.service_account import Credentials
import numpy as np

try: 
    import gspread
except ModuleNotFoundError:
    !{sys.executable} -m pip install gspread
    import gspread
    
try: 
    import pdfplumber
except ModuleNotFoundError:
    !{sys.executable} -m pip install pdfplumber    
    import pdfplumber

## Options

In [2]:
phase = "Phase 3 (Heightened Alert)"
update_google_sheets = True # if True, will update Google Sheet directly

### Config for Google Sheets
(Only required if update_google_sheets is True)

In [3]:
SPREADSHEET_ID = '1gFTNs_GtnTIyyVWXmsQxwdZpGbyicZM2HJcXvCf4b3k'
KEYS_FILE = 'project-gsheets-c30a9f47e838.json'
SHEET_NAME = "Sheet1"
SCOPES = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive'] # If modifying these scopes, delete the file token.pickle.

### More options

In [4]:
# For web scraping
base_url = 'https://www.moh.gov.sg'
landing_page_url = base_url + '/covid-19/situation-report'
past_updates_page_url = 'https://www.moh.gov.sg/covid-19/past-updates'
info_link_string_to_find = "New Cases of Locally Transmitted COVID-19 Infection"
date_tag = "span"
unlinked_string_to_find = " are currently unlinked"
linked_string_to_find1 = " are linked"
linked_string_to_find2 = " is linked"
total_number_pre = "Of the "
total_number_post = " cases"

# PDF tables
tables = ['Figure 1.1', 'Figure 3.1']
headers1 = [
    'Date',
    'Daily Imported',
    'Daily Local transmission',
    'Local cases residing in dorms MOH report',
    'Daily Confirmed'
]
data1_col_index = [0,3,12,16,18]
headers2 = [
    'Date',
    'Intensive Care Unit (ICU)', 
    'General Wards MOH report',
    'In Isolation MOH report', 
    'Total Completed Isolation MOH report',
    'Total Hospital Discharged MOH report',
    'Cumulative Deaths'
]
data2_col_index = [0,1,2,3,4,5,6]
historical_columns = ['False Positives Found', 'Passed but not due to COVID', 'Tested positive demise'] # set all to zero

# Get past data
data_dot_world_url = 'https://query.data.world/s/3hxjhjvlkk7j4re3lzfghvu5sxpw5h'

## Prepare required data

In [5]:
data_dot_world_df = pd.read_excel(data_dot_world_url)
data_dot_world_df.columns = data_dot_world_df.columns.str.strip()
data_dot_world_df['Date'] = pd.to_datetime(data_dot_world_df['Date'])
date = (data_dot_world_df.tail(1).reset_index(drop = True).loc[0, 'Date'] + timedelta(days=1)).strftime("%d %b %Y")
year = datetime.strptime(date, "%d %b %Y").strftime("%Y")

## Web Scraping

In [6]:
def get_info_page_url(date_string):
    past_updates_page = requests.get(past_updates_page_url)
    past_updates_page_soup = BeautifulSoup(past_updates_page.text, "html.parser")
    
    for linebreak in past_updates_page_soup.find_all('br'):
        linebreak.extract()
        
    date_elems_list = past_updates_page_soup.find_all(date_tag, text=re.compile('^.*{}.*$'.format(date_string)))
    
    entries_list = [elem.find_parent("tr") for elem in date_elems_list if info_link_string_to_find in str(elem.find_parent("tr"))]
    
    if len(entries_list) < 2:
        return None

    entry = entries_list[0]

    info_page_url = entry.find(href=True).get("href")
    print(info_page_url)
    
    return info_page_url.strip()

In [7]:
def get_num_of_unlinked_cases(date_string):
    info_page_url = get_info_page_url(date_string)
    info_page = requests.get(info_page_url) 
    info_page_soup = BeautifulSoup(info_page.text, "html.parser")
    
    number_unlinked = re.search('\d+(?={})'.format(unlinked_string_to_find), str(info_page_soup))
    number_linked1 = re.search('\d+(?={})'.format(linked_string_to_find1), str(info_page_soup))
    number_linked2 = re.search('\d+(?={})'.format(linked_string_to_find2), str(info_page_soup))
    total_number = re.search('(?<={})\d+(?={})'.format(total_number_pre, total_number_post), str(info_page_soup))
        
    if not number_unlinked:
        total_linked = 0
        if number_linked1:
            total_linked = total_linked + int(number_linked1.group(0))
        if number_linked2:
            total_linked = total_linked + int(number_linked2.group(0))
        return int(total_number.group(0)) - total_linked
    
    return int(number_unlinked.group(0))

In [8]:
def get_pdf_link(date_string):
    
    landing_page = requests.get(landing_page_url)
    landing_page_soup = BeautifulSoup(landing_page.text, "html.parser")

    pdf_link = landing_page_soup.find("a", text=re.compile('^.*{}.*$'.format(date_string))).get("href")
    print(base_url + pdf_link.strip())
    
    return base_url + pdf_link.strip()

In [9]:
def retrieve_data1(raw_df):
    df = raw_df.iloc[:, lambda raw_df1:data1_col_index]
    df.columns = headers1
    return df

In [10]:
def retrieve_data2(raw_df):
    df = raw_df.iloc[:, lambda raw_df1:data2_col_index]
    df.columns = headers2
    return df

In [11]:
# https://stackoverflow.com/questions/62075033/read-pdf-from-url-to-memory-omitting-saving-file-to-local-file

def get_raw_data(date_string):
    pdf_link = get_pdf_link(date_string)
        
    if not pdf_link:
        print('Unable to get pdf link for {}.'.format(date_string))
        return None, None
        
    http = urllib3.PoolManager()
    temp = io.BytesIO()
    temp.write(http.request("GET", pdf_link).data)

    raw_df1, raw_df2 = pd.DataFrame(), pd.DataFrame()
    
    index = 1

    with pdfplumber.open(temp) as pdf: # pdf = pdfplumber.open(temp) 
        for pdf_page in pdf.pages:
            text = pdf_page.extract_text()
            
            if not any(substring in text for substring in tables):
                continue
                
            table = pdf_page.extract_table()
                        
            if index == 1:
                raw_df1 = pd.DataFrame(table).iloc[[21]].replace(r'^\s*$', np.nan, regex=True)
                raw_df1 = raw_df1.dropna(axis=1).reset_index(drop=True)
                raw_df1.columns = range(raw_df1.shape[1])
                index = 2
                continue
            
            raw_df2 = pd.DataFrame(table).iloc[[23]].replace(r'^\s*$', np.nan, regex=True)
            raw_df2 = raw_df2.dropna(axis=1).reset_index(drop=True)
            raw_df2.columns = range(raw_df2.shape[1])

    return raw_df1, raw_df2

In [12]:
# Consolidate data
current_data = get_raw_data(date)

# Combine retrieved data
current_df = pd.merge(retrieve_data1(current_data[0]), retrieve_data2(current_data[1]), on='Date')
current_df = pd.merge(current_df, pd.DataFrame(0, index=np.arange(1), columns=historical_columns), left_index=True, right_index=True)

store_date = year + datetime.strptime(current_df.loc[0, 'Date'], "%d-%b").strftime("-%m-%d")
current_df = current_df.drop(columns=['Date'])

current_df['Discharged to Isolation'] = current_df['In Isolation MOH report']
#current_df['Unlinked community cases'] = get_num_of_unlinked_cases(date)
current_df['Unlinked community cases'] = 0   ### applicable when no unlinked cases as the sentence structure changes causing function to fail
current_df['Linked community cases'] = int(current_df.loc[0, 'Daily Local transmission']) - int(current_df.loc[0, 'Unlinked community cases'])
current_df['Local cases not residing in doms MOH report'] = current_df['Daily Local transmission']


current_df = current_df.replace(',','', regex=True)

current_df = current_df.apply(pd.to_numeric)

current_df['Phase'] = phase
current_df['Date'] = datetime.strptime(store_date, "%Y-%m-%d")

current_df

https://www.moh.gov.sg/docs/librariesprovider5/local-situation-report/ceg_20210628_daily_report_on_covid-19.pdf?sfvrsn=f3d5f6c_0


,Daily Imported,Daily Local transmission,Local cases residing in dorms MOH report,Daily Confirmed,Intensive Care Unit (ICU),General Wards MOH report,In Isolation MOH report,Total Completed Isolation MOH report,Total Hospital Discharged MOH report,Cumulative Deaths,False Positives Found,Passed but not due to COVID,Tested positive demise,Discharged to Isolation,Unlinked community cases,Linked community cases,Local cases not residing in doms MOH report,Phase,Date
0,5,4,0,9,3,132,170,58570,3642,36,0,0,0,170,0,4,4,Phase 3 (Heightened Alert),2021-06-28


## Combining Data

In [13]:
df = data_dot_world_df.append(current_df, ignore_index = True)
current_index = len(df)-1

In [14]:
# # Check date 
                                             
# if df.loc[current_index, 'Date'] < df.loc[current_index - 1, 'Date']:
#     raise Exception("Current date is earlier than previous entry!")

# if df.loc[current_index, 'Date'] - df.loc[current_index - 1, 'Date'] != timedelta(days=1):
#     raise Exception("Have you skipped a date?")

In [15]:
# Compute extra fields
df.loc[current_index, 'Daily Discharged'] = df.loc[current_index, 'Total Completed Isolation MOH report'] - df.loc[current_index - 1, 'Total Completed Isolation MOH report'] + df.loc[current_index, 'Total Hospital Discharged MOH report'] - df.loc[current_index - 1, 'Total Hospital Discharged MOH report']
df.loc[current_index, 'Cumulative Discharged'] = df['Daily Discharged'].sum()
df.loc[current_index, 'Cumulative Confirmed'] = df['Daily Confirmed'].sum() - df['False Positives Found'].sum()
df.loc[current_index, 'Still Hospitalised'] = df.loc[current_index, 'Cumulative Confirmed'] - df['Daily Discharged'].sum() - df.loc[current_index, 'Cumulative Deaths'] - df.loc[current_index, 'Discharged to Isolation'] - df['Tested positive demise'].sum()
df.loc[current_index, 'Daily Deaths'] = df.loc[current_index, 'Cumulative Deaths'] - df.loc[current_index - 1, 'Cumulative Deaths']
pd.set_option('display.max_columns', None)
df.tail(5)

,Date,Daily Confirmed,False Positives Found,Cumulative Confirmed,Daily Discharged,Passed but not due to COVID,Cumulative Discharged,Discharged to Isolation,Still Hospitalised,Daily Deaths,Cumulative Deaths,Tested positive demise,Daily Imported,Daily Local transmission,Local cases residing in dorms MOH report,Local cases not residing in doms MOH report,Intensive Care Unit (ICU),General Wards MOH report,In Isolation MOH report,Total Completed Isolation MOH report,Total Hospital Discharged MOH report,Linked community cases,Unlinked community cases,Phase
518,2021-06-24,23,0.0,62493.0,27.0,0,62162.0,182,106.0,0.0,35,0,9,14,0.0,14.0,4,132.0,182.0,58517.0,3623.0,12.0,2.0,Phase 3 (Heightened Alert)
519,2021-06-25,20,0.0,62513.0,21.0,0,62183.0,178,109.0,0.0,35,0,5,15,0.0,15.0,4,135.0,178.0,58532.0,3629.0,12.0,3.0,Phase 3 (Heightened Alert)
520,2021-06-26,17,0.0,62530.0,20.0,0,62203.0,170,113.0,1.0,36,0,4,13,0.0,13.0,4,139.0,170.0,58543.0,3638.0,9.0,4.0,Phase 3 (Heightened Alert)
521,2021-06-27,14,0.0,62544.0,14.0,0,62217.0,165,118.0,0.0,36,0,2,12,0.0,12.0,4,144.0,165.0,58555.0,3640.0,11.0,1.0,Phase 3 (Heightened Alert)
522,2021-06-28,9,0.0,62553.0,17.0,0,62234.0,170,105.0,0.0,36,0,5,4,0.0,4.0,3,132.0,170.0,58570.0,3642.0,4.0,0.0,Phase 3 (Heightened Alert)


## Push new data to Google sheet

In [16]:
if update_google_sheets:

    credentials = Credentials.from_service_account_file(KEYS_FILE, scopes=SCOPES)
    gc = gspread.authorize(credentials)

    ws = gc.open_by_key(SPREADSHEET_ID).worksheet(SHEET_NAME)

    df['Date'] = df['Date'].astype(str)
    df.fillna('', inplace=True)

    ws.update([df.columns.values.tolist()] + df.values.tolist())